In [ ]:
root_dir = "output"
method_name = "standard"
benchmark = "tabmwp"

model = "gpt-3.5-turbo"
eval_model = "gpt-4o-mini"
seed = 42
num_retries = 1
warming = [0.0]

In [ ]:
from agential.prompting.standard.prompting import Standard
method = Standard(
    llm=llm,
    benchmark=benchmark,
)

In [3]:
from datasets import load_dataset

/opt/anaconda3/envs/agential/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
data = load_dataset("google-research-datasets/mbpp")['test']

Generating prompt split: 100%|██████████| 10/10 [00:00<00:00, 4534.87 examples/s]


In [9]:
from curses.ascii import EM
from agential.utils.general import safe_execute

eval_table_data = []
perf_table_data = []
em_scores = []
outputs = []
index = 0 
for instance in data:
    if index > 9:
        break
    question = instance['text']
    answer = instance['code']
    seq = '\n'.join(instance['test_list'])
    
    # Building the prompt
    question = f"""You are an expert Python programmer, and here is your task:
Write a function to find the shared elements from the given two lists.
Your code should pass these tests:

{seq}"""
    
    # Inference
    out = method.generate(
        question=question,
        key=answer,
        num_retries=num_retries,
        warming=warming
    )

    code_str = out.answer.replace("```python", "").replace("```", "").strip()
    _, execution_status = safe_execute(
        f"from typing import *\n\n{code_str}\n{answer}"
    )
    
    is_correct = int(EM(execution_status, "Done", normalize=False))


    # Update scores.
    em_scores.append(is_correct)
    # Update tables.
    eval_table_data.append([question, answer, execution_status, out.answer, is_correct])
    perf_table_data.append([
        out.total_prompt_tokens, 
        out.total_completion_tokens, 
        out.total_tokens, 
        out.total_prompt_cost,
        out.total_completion_cost,
        out.total_cost,
        out.total_prompt_time,
        out.total_time
    ])
    # Update outputs.
    outputs.append(out)
    # Log metrics per instance.
    run.log({
        "pass@k=1": is_correct,
    })
    
    


  
    

Question:  You are an expert Python programmer, and here is your task:
Write a function to find the shared elements from the given two lists.
Your code should pass these tests:

assert remove_Occ("hello","l") == "heo"
assert remove_Occ("abcda","a") == "bcd"
assert remove_Occ("PHP","P") == "H"

Question:  You are an expert Python programmer, and here is your task:
Write a function to find the shared elements from the given two lists.
Your code should pass these tests:

assert sort_matrix([[1, 2, 3], [2, 4, 5], [1, 1, 1]])==[[1, 1, 1], [1, 2, 3], [2, 4, 5]]
assert sort_matrix([[1, 2, 3], [-2, 4, -5], [1, -1, 1]])==[[-2, 4, -5], [1, -1, 1], [1, 2, 3]]
assert sort_matrix([[5,8,9],[6,4,3],[2,1,4]])==[[2, 1, 4], [6, 4, 3], [5, 8, 9]]

Question:  You are an expert Python programmer, and here is your task:
Write a function to find the shared elements from the given two lists.
Your code should pass these tests:

assert count_common(['red','green','black','pink','black','white','black','eyes','w

In [10]:
"""You are an expert Python programmer, and here is your task: Write a function to find the shared elements from the given two lists.
Your code should pass these tests.

```python
def similar_elements(test_tup1, test_tup2):
    return tuple(set(test_tup1) | set(test_tup2))

assert set(similar_elements((3, 4, 5, 6), (5, 7, 4, 10))) == set((4, 5))
assert set(similar_elements((1, 2, 3, 4), (5, 4, 3, 7))) == set((3, 4))
assert set(similar_elements((11, 12, 14, 13), (17, 15, 14, 13))) == set((13, 14))
```"""


'You are an expert Python programmer, and here is your task: Write a function to find the shared elements from the given two lists.\nYour code should pass these tests.\n\n```python\ndef similar_elements(test_tup1, test_tup2):\n    return tuple(set(test_tup1) | set(test_tup2))\n\nassert set(similar_elements((3, 4, 5, 6), (5, 7, 4, 10))) == set((4, 5))\nassert set(similar_elements((1, 2, 3, 4), (5, 4, 3, 7))) == set((3, 4))\nassert set(similar_elements((11, 12, 14, 13), (17, 15, 14, 13))) == set((13, 14))\n```'